# Training

In [2]:
# !pip install -U efficientnet

     |████████████████████████████████| 50 kB 733 kB/s eta 0:00:01


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import random
import sys
import sklearn as sk
from IPython.display import Image, display
from pathlib import Path
from tqdm import tqdm
import pickle
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
import math
import re
import efficientnet.tfkeras as efn

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.models import model_from_json

In [4]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.4.1
Keras Version: 2.4.0
Python 3.7.9 | packaged by conda-forge | (default, Feb 13 2021, 20:03:11) 
[GCC 9.3.0]
Pandas 1.1.5
Scikit-Learn 0.24.1
GPU is available


In [5]:
AUTO = tf.data.experimental.AUTOTUNE
strategy = tf.distribute.get_strategy()

BATCH_SIZE = 32
IMAGE_SIZE = [224, 224]
SEED = 42
LR = 0.001
VERBOSE = 1
N_CLASSES = 11014
FOLDS = 5

In [6]:
# IMAGE_SIZE

[224, 224]

In [7]:
TRAINING_FILENAMES = tf.io.gfile.glob('../input/shopee-tf-records-512-stratified' + '/*.tfrec')
TRAINING_FILENAMES

['../input/shopee-tf-records-512-stratified/train02-2284.tfrec',
 '../input/shopee-tf-records-512-stratified/train08-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train10-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train05-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train01-2284.tfrec',
 '../input/shopee-tf-records-512-stratified/train12-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train09-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train07-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train00-2284.tfrec',
 '../input/shopee-tf-records-512-stratified/train06-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train03-2284.tfrec',
 '../input/shopee-tf-records-512-stratified/train11-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train04-2284.tfrec',
 '../input/shopee-tf-records-512-stratified/train13-2283.tfrec',
 '../input/shopee-tf-records-512-stratified/train14-2283.tfrec']

In [8]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [9]:
def arcface_format(posting_id, image, label_group, matches):
    return posting_id, {'inp1': image, 'inp2': label_group}, label_group, matches

In [10]:
def data_augment(posting_id, image, label_group, matches):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.70, 1.30)
    image = tf.image.random_contrast(image, 0.80, 1.20)
    image = tf.image.random_brightness(image, 0.10)
    return posting_id, image, label_group, matches

In [11]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [12]:
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "posting_id": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "label_group": tf.io.FixedLenFeature([], tf.int64),
        "matches": tf.io.FixedLenFeature([], tf.string)
    }

    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    posting_id = example['posting_id']
    image = decode_image(example['image'])
    label_group = tf.cast(example['label_group'], tf.int32)
    matches = example['matches']
    return posting_id, image, label_group, matches

In [13]:
def load_dataset(filenames, ordered = False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_labeled_tfrecord, num_parallel_calls = AUTO) 
    return dataset

In [14]:
def get_training_dataset(filenames, ordered = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [15]:
def get_validation_dataset(filenames, ordered = True):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset

In [16]:
def count_data_items(filenames):
    # The number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
print(f'Dataset: {NUM_TRAINING_IMAGES} training images')

Dataset: 34250 training images


In [17]:
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

In [18]:
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [19]:
def get_model():
    with strategy.scope():
        margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.5, 
            name='head/arc_margin', 
            dtype='float32'
            )

        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x = efn.EfficientNetB3(weights = 'imagenet', include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = margin([x, label])
        
        output = tf.keras.layers.Softmax(dtype='float32')(x)

        model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
        
        opt = tf.keras.optimizers.Adam(learning_rate = LR)

        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        
        return model

In [20]:
def train_and_evaluate(intial_ep = 0, epoch = 20):

    # Seed everything
    seed_everything(SEED)
    train, valid = train_test_split(TRAINING_FILENAMES, shuffle = True, random_state = SEED)
    train_dataset = get_training_dataset(train, ordered = False)
    train_dataset = train_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    val_dataset = get_validation_dataset(valid, ordered = True)
    val_dataset = val_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    STEPS_PER_EPOCH = count_data_items(train) // BATCH_SIZE
    K.clear_session()
    model = get_model()
#     model.load_weights('../input/shopee/EfficientNetB3_224_42_score_0.71.h5')
    model.summary()
    
    # Model checkpoint
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'EfficientNetB3_{IMAGE_SIZE[0]}_{SEED}.h5', 
                                                    monitor = 'val_loss', 
                                                    verbose = VERBOSE, 
                                                    save_best_only = True,
                                                    save_weights_only = True, 
                                                    mode = 'min')

    history = model.fit(train_dataset,
                        steps_per_epoch = STEPS_PER_EPOCH,
                        initial_epoch=intial_ep,
                        epochs = epoch,
                        callbacks = [checkpoint, get_lr_callback()], 
                        validation_data = val_dataset,
                        verbose = VERBOSE)

In [21]:
train_and_evaluate(0, 40)

44113920/44107200 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp1 (InputLayer)               [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-b3 (Functional)    (None, None, None, 1 10783528    inp1[0][0]                       
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 1536)         0           efficientnet-b3[0][0]            
__________________________________________________________________________________________________
inp2 (InputLayer)               [(None,)]            0                                            
_____________________________

In [22]:
def train_and_evaluate(intial_ep = 0, epoch = 20):
    # Seed everything
    seed_everything(SEED)
    train, valid = train_test_split(TRAINING_FILENAMES, shuffle = True, random_state = SEED)
    train_dataset = get_training_dataset(train, ordered = False)
    train_dataset = train_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    val_dataset = get_validation_dataset(valid, ordered = True)
    val_dataset = val_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    STEPS_PER_EPOCH = count_data_items(train) // BATCH_SIZE
    model = get_model()
    model.load_weights('./EfficientNetB3_224_42.h5')
    model.summary()
    
    # Model checkpoint
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'EfficientNetB3_{IMAGE_SIZE[0]}_{SEED}.h5', 
                                                    monitor = 'val_loss', 
                                                    verbose = VERBOSE, 
                                                    save_best_only = True,
                                                    save_weights_only = True, 
                                                    mode = 'min')

    history = model.fit(train_dataset,
                        steps_per_epoch = STEPS_PER_EPOCH,
                        initial_epoch=intial_ep,
                        epochs = epoch,
                        callbacks = [checkpoint, get_lr_callback()], 
                        validation_data = val_dataset,
                        verbose = VERBOSE)

train_and_evaluate(40, 50)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp1 (InputLayer)               [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-b3 (Functional)    (None, None, None, 1 10783528    inp1[0][0]                       
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 1536)         0           efficientnet-b3[0][0]            
__________________________________________________________________________________________________
inp2 (InputLayer)               [(None,)]            0                                            
____________________________________________________________________________________________